# Training

In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

In [ ]:
import random
import wandb
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import os
from datetime import datetime
import tensorflow.keras.layers as tfkl
import tensorflow.keras as tfk 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from wandb.keras import WandbCallback

from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_unet_collection import models
import keras_unet_collection.losses as kul
import tensorflow.keras.losses as tfkloss
import tensorflow.keras.backend as K

# from UnetsMM.vanilla_multimodelU import MultiModalNet

enable dynamic memory

In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)

Tensorflow checks

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Setting default parameters

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# plots sizes
plt.rcParams['figure.figsize'] = [9, 9]

config

In [ ]:
config = {"epochs": 150,
          "batch_size": 64,
          "input_shape": (None, None, 1),
          "filters": [32, 64, 128, 256],
          'optimizer': 'adam',
          'loss': kul.tversky,
          'val_split': 0.15,
          'metrics': ["BinaryAccuracy", "Recall", "Precision", ], 
          'dataset': 'lorenzob/Thesis/TR_img2-img3_eq:v1',
          'model': 'UNet Long'
            }

## Define functions

UNet

In [ ]:
class UNet():
    def __init__(self, input_shape, activ_encod = 'ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='glorot_uniform'):
        # define activation functions
        self.activ_encod = activ_encod
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # layer to be used in the network creation
        self.encoder = self.input
        self.decoder = None
        
        # list where I save all conv layers that will be concatenated through
        # the skip connection. This will contain the 2 list of the pool layers
        # of the 2 encoders
        self.pool_layers_list = []

    def Down_Conv_block(self, inp, filters, encoder, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        pool = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)
        self.pool_layers_list.append(conv2)
        return pool

    def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(conv2)
        concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
        return concat


    def build_model(self, filters_list):

        # Encoder 
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder = self.Down_Conv_block(self.encoder, filters, encoder=0, activ=self.activ_encod)

        # reverse the list of layers to give to the encoder in the right order
        rev_list = self.pool_layers_list[::-1]

        # set the starting layer of the decoder
        self.decoder = self.encoder

        # Decoder
        for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
            self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
        # first convolutions of filters_list
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)

        # custom convolutions
        # layer = tfkl.Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)
        # layer = tfkl.Conv2D(filters=2, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)

        # output
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

UNeXt

In [ ]:
class UNeXt():
    def __init__(self, input_shape, activ_encod = 'gelu', activ_decod='gelu', activ_out='sigmoid', kern_init='HeUniform'):
        # define activation functions
        self.activ_encod = activ_encod
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # layer to be used in the network creation
        self.encoder = self.input
        self.decoder = None
        
        # list where I save all conv layers that will be concatenated through
        # the skip connection. This will contain the 2 list of the pool layers
        # of the 2 encoders
        self.pool_layers_list = []

    def Down_Conv_block(self, inp, filters, encoder, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=7, strides=1, padding='same', activation=None, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters*4, kernel_size=1, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        conv3 = tfkl.Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', activation=None, kernel_initializer=self.kern_init)(conv2)
        sum_4 = tfkl.Add()([conv1, conv3])
        pool = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(sum_4)
        self.pool_layers_list.append(sum_4)
        return pool

    def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=7, strides=1, padding='same', activation=None, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters*4, kernel_size=1, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        conv3 = tfkl.Conv2D(filters=filters, kernel_size=1, strides=1, padding='same', activation=None, kernel_initializer=self.kern_init)(conv2)
        sum_4 = tfkl.Add()([conv1, conv3])
        up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(sum_4)
        concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
        return concat


    def build_model(self, filters_list):

        # Encoder 
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder = self.Down_Conv_block(self.encoder, filters, encoder=0, activ=self.activ_encod)

        # reverse the list of layers to give to the encoder in the right order
        rev_list = self.pool_layers_list[::-1]

        # set the starting layer of the decoder
        self.decoder = self.encoder

        # Decoder
        for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
            self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
        # first convolutions of filters_list
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)

        # output
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

FreezedNet

In [ ]:
class FreezedNet():
    def __init__(self, trained_model, input_shape, activ_encod_1='ReLU', activ_encod_2='ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='HeNormal'):
        # define activation functions
        self.activ_encod_1 = activ_encod_1
        self.activ_encod_2 = activ_encod_2
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # setting the UNet model to "non training mode"
        trained_model.trainable = False

        # layer to be used in the network creation
        self.tr_model = trained_model(self.input[ :, :, :, 0:1], training=False)
        self.encoder = self.input[ :, :, :, 1:2]
        self.decoder = None

    def Down_Conv_block(self, inp, filters, activ, k_size=3):
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv)   
        return conv

    def Up_Conv_block(self, inp, filters, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        
        return conv2


    def build_model(self, filters_list):
        
        # Encoder nuclei
        for filters in filters_list:
            self.encoder = self.Down_Conv_block(self.encoder, filters, activ=self.activ_encod_1)

        # asign decoder
        self.decoder = self.encoder

        # Decoder Nuclei
        for filters in filters_list[-2::-1]:
            self.decoder = self.Up_Conv_block(self.decoder, filters, activ=self.activ_decod)

        # 2D Nuclei image
        self.decoder = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        self.decoder = tfkl.Conv2D(filters=1, kernel_size=1, strides=1, activation='sigmoid', kernel_initializer=self.kern_init)(self.decoder)
        # output
        layer = tfkl.Multiply()([self.tr_model, self.decoder])

        # last convolutions

        # Encoder 
        for filters in filters_list:
            layer = self.Down_Conv_block(layer, filters, activ=self.activ_encod_1)

        # Decoder 
        for filters in filters_list[-2::-1]:
            layer = self.Up_Conv_block(layer, filters, activ=self.activ_decod)
        
        #  output        
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid', kernel_initializer=self.kern_init)(layer)
        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

Function to download data

In [ ]:
def download_data(project_name, artifact_path, entity='lorenzob', img_filename='images.npy', ann_filename='annotations.npy'):

    with wandb.init(entity=entity, project=project_name, job_type="downloading_data") as run:

        # we first retrive the artifact object
        artifact = run.use_artifact(artifact_path)

        # we then assign to a variable the path to the wanted file
        path_x = artifact.get_path(img_filename)
        path_y = artifact.get_path(ann_filename)

        # we load the binary array into a 'normal' one with this function
        X = np.load(path_x.download())
        Y = np.load(path_y.download())

    return X, Y

## Importing, rescaling and train-test splitting X and Y

In [ ]:
wandb.login()

In [ ]:
X, Y = download_data('Thesis', config['dataset'])

In [ ]:
X = X[:, :, :, 0:1]

In [ ]:
print(X.shape, Y.shape)
print(np.min(X), np.max(X), np.min(Y), np.max(Y))

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=config['val_split'], random_state=seed)

Compute steps per epoch

In [ ]:
steps_per_epoch = np.ceil(X_train.shape[0]/config['batch_size'])
steps_per_epoch

## Data generators

In [ ]:
## FOR TRAIN DATA

image_datagen_train = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    dtype=np.float32)
    # rotation_range=10,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # fill_mode='reflect',)


mask_datagen_train = ImageDataGenerator(
    # preprocessing_function=np.round, # to round all 'creted' pixels of the annotation
    horizontal_flip=True,
    vertical_flip=True,
    dtype=np.float32)
    # rotation_range=10,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # fill_mode='reflect',)


image_generator_train = image_datagen_train.flow(X_train, batch_size=config['batch_size'], seed=seed)
mask_generator_train = mask_datagen_train.flow(Y_train, batch_size=config['batch_size'], seed=seed)

train_generator = zip(image_generator_train, mask_generator_train)

## FOR VALIDATION DATA

datagen_val = ImageDataGenerator(dtype=np.float32)

val_generator = datagen_val.flow(X_val, Y_val, batch_size=8, seed=seed)

## Model

In [ ]:
# unet = tf.keras.models.load_model(f"models/eq_64.h5",  custom_objects={'tversky': config['loss']})
# # loaded_model. build()

In [ ]:
model = UNet(config['input_shape'], kern_init='HeUniform').build_model(config['filters'])

In [ ]:
# model = FreezedNet(unet, config['input_shape']).build_model(config['filters'])

In [ ]:
model.compile(optimizer=config['optimizer'], loss=config['loss'], metrics=config['metrics'])

In [ ]:
model.summary()
# tfk.utils.plot_model(model)

In [ ]:
del X, X_train, X_val, Y, Y_train, Y_val, datagen_val, image_datagen_train, image_generator_train, mask_datagen_train, mask_generator_train

## Starting the W&B run

In [ ]:
# run_name = 'R_'+ datetime.now().strftime('%m%d_%H%M%S')
run = wandb.init(project="Thesis", entity="lorenzob", notes=f"img2-3, DA, ann2.1-3.1", job_type='training',
           config=config)

### Training

In [ ]:
# wandb_callback = WandbCallback(monitor='val_loss')
reduce_lr = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=7, min_lr=0)
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, 
    verbose=0, mode='auto',
    restore_best_weights=True
)
callabacks = [reduce_lr, earlystopper,WandbCallback(),]#, SemanticLogger(ind)

In [ ]:
history = model.fit(
    x = train_generator,
    epochs = config['epochs'],
    validation_data=val_generator,
    callbacks = callabacks,
    steps_per_epoch=steps_per_epoch
).history

In [ ]:
wandb.finish()

Save the model

In [ ]:
# !nvidia-smi

In [ ]:
# model.save('models/img3_UNetLo')